In [2]:
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import scanpy as sc
import anndata
import os
import sys
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

In [3]:
sys.path.insert(1, '/secure/projects/tacco/')
import tacco as tc

In [4]:
sys.path.insert(1, '/secure/projects/HTAPP_MBC/src/spatial')
from spatial_utils import *

In [5]:
plots_dir=out_dir_base+"/plots/"
os.makedirs(plots_dir, exist_ok=True)

In [6]:
sample_sheet=pd.read_csv(codeDir+"/meta/sampletracking_HTAPP_MBC_spatial.csv")

In [7]:
#These identifiers are set as defaults in many functions. If names are changed, this has to be adapted when calling the functions.
ct_column='cell_type'
#sm_methods=["exseq","exseq_bin","slide_seq","merfish","merfish_bin","codex"] #spatial mathod names are hard-coded for now because some methods need special treatments. Can be fixed by giving eatch method it's own variable.
sm_methods=["slide_seq","merfish","merfish_bin","exseq","exseq_bin","codex"] #spatial mathod names are hard-coded for now because some methods need special treatments. Can be fixed by giving eatch method it's own variable.

sc_method="scRNAseq"
all_methods=[sc_method,*sm_methods]
all_methods

['scRNAseq',
 'slide_seq',
 'merfish',
 'merfish_bin',
 'exseq',
 'exseq_bin',
 'codex']

In [8]:
run_samples=list(filter(lambda a: not a in [''], list(sample_sheet['sample'].unique()))) 
run_samples

['HTAPP-944-SMP-7479',
 'HTAPP-895-SMP-7359',
 'HTAPP-880-SMP-7179',
 'HTAPP-878-SMP-7149',
 'HTAPP-812-SMP-8239',
 'HTAPP-514-SMP-6760',
 'HTAPP-982-SMP-7629',
 'HTAPP-997-SMP-7789',
 'HTAPP-917-SMP-4531',
 'HTAPP-853-SMP-4381',
 'HTAPP-313-SMP-932',
 'HTAPP-330-SMP-1082',
 'HTAPP-783-SMP-4081',
 'HTAPP-364-SMP-1321',
 'HTAPP-213-SMP-6752']

In [ ]:
for sample in run_samples:
    print(sample)
    out_dir=out_dir_base+sample 
    adatas=get_and_process_data(sample,data_dir,out_dir,methods=all_methods,process=False,verbose=False)
    adatas=get_squares(adatas)
    update_processed_adatas(adatas,sample,out_dir,sm_methods,overwrite=True,process=False)

In [ ]:
for sample in run_samples:
    print(sample)
    out_dir=out_dir_base+sample 
    adatas=get_and_process_data(sample,data_dir,out_dir,methods=all_methods,process=False,verbose=False)
    sel_colors=colors[colors.index.isin(adatas[sc_method].obs[ct_column])]
    fig=tc.pl.scatter(square(replicate_splitter(adatas[~adatas.index.isin(['scRNAseq'])])), keys=['RCTD','OT','OT_max'],
                      colors=sel_colors,joint=True,padding=0.8);
    figheight = fig.get_size_inches()[1]
    x,y = fig.axes[0].transData.inverted().transform(fig.axes[0].transAxes.transform((0,1))+np.array([0,100]))
    fig.axes[0].text(x, y, sample, ha="left", va="bottom", fontsize='large')
    for ax in fig.get_axes():
        scalebar = AnchoredSizeBar(ax.transData, 100, '100µm', 'lower left', pad=0.3, color='black', frameon=False, 
                                   size_vertical=2, fontproperties=fm.FontProperties(size=10))
        ax.add_artist(scalebar)
    fig.savefig(f'{plots_dir}{sample}_ROI.pdf',bbox_inches='tight');
    #can't save when plots are shown in jl --> cleared output (first sample as example)

In [ ]:
for sample in run_samples:
    print(sample)
    out_dir=out_dir_base+sample 
    adatas=get_and_process_data(sample,data_dir,out_dir,methods=all_methods,process=False,verbose=False)
    sel_colors=colors[colors.index.isin(adatas[sc_method].obs[ct_column])]
    fig = tc.pl.scatter(replicate_splitter(adatas[~adatas.index.isin(['scRNAseq'])]), keys=['RCTD','OT','OT_max'],
                        colors=sel_colors,joint=True,sharex='all',sharey='all',axsize=1.5e-3,point_size=2, padding=0.4,margin=0.2);
    figheight = fig.get_size_inches()[1]
    x,y = fig.axes[0].transData.inverted().transform(fig.axes[0].transAxes.transform((0,1))+np.array([0,100]))
    fig.axes[0].text(x, y, sample, ha="left", va="bottom", fontsize='large')
    for ax in fig.get_axes():
        scalebar = AnchoredSizeBar(ax.transData, 100, '100µm', 'lower left', pad=0.3, color='black', frameon=False, 
                                   size_vertical=2, fontproperties=fm.FontProperties(size=10))
        ax.add_artist(scalebar)
    fig.savefig(f'{plots_dir}{sample}_spatial_scatter.pdf',bbox_inches='tight')
    
    #can't save when plots are shown in jl --> cleared output (first sample as example)

In [ ]:
fig,axs=tc.pl.subplots(len(run_samples),len(sm_methods),sharex='col',sharey='col', x_padding=0.4,y_padding=0.4, width_ratios=[0.3,1,1,0.7,0.2,1,0.2,0.2,0.3,0.2,0.2,0.2,0.6,0.2,1])

for i_sample,sample in enumerate(run_samples):
    print(sample)
    out_dir=out_dir_base+sample 
    adatas=get_and_process_data(sample,data_dir,out_dir,methods=all_methods,process=False,verbose=False)
    sel_colors=colors[colors.index.isin(adatas[sc_method].obs[ct_column])]
    
    method_axes = []
    method_adatas = {}
    for i_method,method in enumerate(sm_methods):
        
        ax=axs[i_method,i_sample]
        sel_adatas=replicate_splitter(adatas[adatas.index.isin([method])])
        ax.axis("off")
        if len(sel_adatas)!=0:
            method_axes.append(ax)
            method_adatas[method] = sel_adatas.iloc[0]
    
    tc.pl.scatter(method_adatas, keys=['OT'],method_labels={'OT':''},
                        colors=sel_colors,joint=True,sharex='all',sharey='all',point_size=2,margin=0.2,ax=np.array(method_axes)[None,:], grid=False, legend=False, noticks=True);
    
    for ax in method_axes:
        scalebar = AnchoredSizeBar(ax.transData, 1000, '1mm', 'lower left', pad=0.3, color='black', frameon=False, 
                                   size_vertical=2, fontproperties=fm.FontProperties(size=10))
        ax.add_artist(scalebar)
        ax.set_title("")
    
    ax=axs[0,i_sample]
    figheight = fig.get_size_inches()[1]
    x,y = ax.transData.inverted().transform(ax.transAxes.transform((0,1))+np.array([0,100]))
    ax.text(x, y, sample, ha="left", va="bottom", fontsize='large')
    
fig.savefig(f'{plots_dir}joint_spatial_scatter.pdf',bbox_inches='tight')

#can't save when plots are shown in jl --> cleared output (first sample as example)

HTAPP-944-SMP-7479


/secure/projects/tacco/tacco/plots/_plots.py:872: RuntimeWarning: invalid value encountered in true_divide
  weights = weights / weights.sum(axis=-1)[...,None]


HTAPP-895-SMP-7359


/secure/projects/tacco/tacco/plots/_plots.py:872: RuntimeWarning: invalid value encountered in true_divide
  weights = weights / weights.sum(axis=-1)[...,None]


HTAPP-880-SMP-7179


...

/secure/projects/tacco/tacco/plots/_plots.py:872: RuntimeWarning: invalid value encountered in true_divide
  weights = weights / weights.sum(axis=-1)[...,None]


HTAPP-364-SMP-1321


/secure/projects/tacco/tacco/plots/_plots.py:872: RuntimeWarning: invalid value encountered in true_divide
  weights = weights / weights.sum(axis=-1)[...,None]


HTAPP-213-SMP-6752


/secure/projects/tacco/tacco/plots/_plots.py:872: RuntimeWarning: invalid value encountered in true_divide
  weights = weights / weights.sum(axis=-1)[...,None]
